In [1]:

%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import newHvArray as hv
import iopro
import numpy as np
import itertools
import array
import ROOT
import rootpy as rpy
import root_numpy as rnp
from datetime import datetime


/phys/users/tommeyer/Resources/miniconda/envs/HV/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
np.set_printoptions(suppress=True, linewidth=100)

In [17]:
valgrind

NameError: name 'valgrind' is not defined

In [4]:
#trk: 1-3 | jet: 4-6  | 
emFrac = "jet_AntiKt4LCTopo_emfrac" #|0
trkPt  = "trk_pt"                   #|1
trkPhi = "trk_phi_wrtPV"            #|2
trkEta = "trk_eta"                  #|3

jetPt  = "jet_AntiKt4LCTopo_pt"     #|4
jetPhi = "jet_AntiKt4LCTopo_phi"    #|5
jetEta = "jet_AntiKt4LCTopo_eta"    #|6

child  = "mc_child_index"           #|7
pdgId  = "mc_pdgId"                 #|8
vx_x   = "mc_vx_x"                  #|9
vx_y   = "mc_vx_y"                  #|10
mc_eta = "mc_eta"                   #|11
mc_phi = "mc_phi"                   #|12
#-----
    
branchNames = [ emFrac,
trkPt  ,
trkPhi ,
trkEta ,
jetPt  ,
jetPhi ,
jetEta ,
child  ,
pdgId  ,
vx_x   ,
vx_y   ,
mc_eta ,
mc_phi ]

In [5]:
def loader(loadChoice):

    bkgFolderName = "mc12_8TeV.147912.Pythia8_AU2CT10_jetjet_JZ2W.merge.NTUP_COMMON.e1126_s1469_s1470_r6262_p1575_tid05435083_00"
    bkgFilePath   = "/phys/groups/tev/scratch4/users/gwatts/GRIDDS/{0}/{0}".format(bkgFolderName)
    bkgFileName   = "NTUP_COMMON.05435083._*"

    sigFilePath = "/phys/groups/tev/scratch3/users/HV/WHHV/ntup_001"
    #sigFileName = "ntup_aod_*"
    sigFileName = "ntup_aod_00_*"
    
    sigFiles = "{0}/{1}".format(sigFilePath, sigFileName)
    bkgFiles = "{0}/{1}".format(bkgFilePath, bkgFileName)

    # Setup Arrays
    if loadChoice.lower() == "signal":
        global sigArray
        sigArray = rnp.root2array( sigFiles, "physics", branchNames)
        return sigArray
    elif loadChoice.lower() == "background":
        global bkgArray
        bkgArray = rnp.root2array( bkgFiles, "physics", branchNames)
        return bkgArray
    else:
        print "No data type given, specify 'signal' or 'background' in arguments"
        sys.exit()

In [6]:
def vert_jetTrk(arrayIN):
    #| trk: 1-3 | jet: 4-6  | 
    #| Jet Length > Trk Length
    #| Calculate eventID, JetID, and restructure Jet data 
    #| Data is restructured to make each row a jet rather than event
   
    #| EventID array: event number for trk/jets
    #| entry = same per event ; len = trk len
    
    #| jets are contained within Tracks
    #| track tracklength is number of jets in event
    
    #| JetID array: track number in event (i.e. event 1 - track 5)
    #| entry = 1 to trk len ; len = trk len
    
    #| Jet arrays: get event data, resize to match trk length
    #|  arbitrarily set values of resized jet arrays to < -500 >
    
    eventID = jetID = jetPt = jetPhi = jetEta = emfrac = np.zeros(1)
    count = 0
    for trkEv, jetptEv, jetphiEv, jetetaEv, emfracEv in itertools.izip_longest(arrayIN[branchNames[1]], arrayIN[branchNames[4]], 
                                                                               arrayIN[branchNames[5]], arrayIN[branchNames[6]], 
                                                                               arrayIN[branchNames[0]] ):
        
        #all var of same class(jet/track) have same length
        trklen = len(trkEv)
        jetlen = len(jetptEv)
        #Ignore any events with no trks
        if trklen != 0:
            #Make jet arrays match trk array length
            emfracEv = np.resize(emfracEv, (1,trklen))
            jetptEv  = np.resize(jetptEv , (1,trklen))
            jetphiEv = np.resize(jetphiEv, (1,trklen))
            jetetaEv = np.resize(jetetaEv, (1,trklen))

            #set extra spaces to arbitrary value
            jetptEv[0][jetlen:] = jetphiEv[0][jetlen:] = jetetaEv[0][jetlen:]  = emfracEv[0][jetlen:] = -500
            #print jetptEv[0]
            emfrac = np.append(emfrac, emfracEv)
            jetPt  = np.append(jetPt , jetptEv )
            jetPhi = np.append(jetPhi, jetphiEv)
            jetEta = np.append(jetEta, jetetaEv)

            eventID = np.append(eventID, [count]*trklen)
            jetID   = np.append(jetID, np.arange(1, trklen+1))
            count += 1
            #del trkEv, jetphiEv, jetptEv, jetetaEv, emfracEv
    
    #remove initial blank entry
    eventID = np.delete(eventID, 0)
    emfrac  = np.delete(emfrac , 0)
    jetID   = np.delete(jetID  , 0)
    jetPt   = np.delete(jetPt  , 0)
    jetPhi  = np.delete(jetPhi , 0)
    jetEta  = np.delete(jetEta , 0)
    
    #Orient arrays as columns
    emfrac = emfrac[None].T
    jetPt  = jetPt [None].T
    jetPhi = jetPhi[None].T
    jetEta = jetEta[None].T
    trkPt  = np.hstack( arrayIN[branchNames[1] ] )[None].T
    trkPhi = np.hstack( arrayIN[branchNames[2] ] )[None].T 
    trkEta = np.hstack( arrayIN[branchNames[3] ] )[None].T
    
    eventID = eventID[None].T
    jetID   = jetID[None].T
    
    data = np.column_stack([eventID, jetID, emfrac, jetPt, jetEta, jetPhi, trkPt, trkPhi, trkEta])
    arrayInfo(data)
    
    del eventID, jetID, trkPt, trkPhi, trkEta, jetPt, jetEta, jetPhi, emfrac
    return data

def vert_mcdata(arrayIN):  
    child  = np.hstack( arrayIN[ branchNames[7] ] )[None].T
    pdgId  = np.hstack( arrayIN[ branchNames[8] ] )[None].T 
    vxX    = np.hstack( arrayIN[ branchNames[9] ] )[None].T  
    vxY    = np.hstack( arrayIN[ branchNames[10]] )[None].T  
    mc_eta = np.hstack( arrayIN[ branchNames[11]] )[None].T  
    mc_phi = np.hstack( arrayIN[ branchNames[12]] )[None].T 
    
    data = np.column_stack([child, pdgId, vxX, vxY, mc_eta, mc_phi])
    arrayInfo(data)
    
    del child, pdgId, vxX, vxY, mc_eta, mc_phi
    return data

def locateHiggsEvents(arrayIN):
    #| Return array of rows containing Higgs events (id 36)
    higgsRows = np.where(arrayIN[: , 1] == 36)[0]
    #| Replace with only rows containing Higgs events
    arrayIN = arrayIN[higgsRows]
    return arrayIN

#######

def arrayInfo(tempArray):
    print tempArray.dtype
    print type(tempArray)
    print "shape: {0}".format(tempArray.shape) 
    print "entries: {0}\n".format(tempArray.size) 
    
def signalTChain():
    f = ROOT.TChain("physics")
    f.Add("/phys/groups/tev/scratch3/users/HV/WHHV/ntup_001/ntup_aod_00_1_50.root")
    for i in range(1,5):
        f1.Add("/phys/groups/tev/scratch3/users/HV/WHHV/ntup_001/ntup_aod_00_{0}*.root".format(i))
    return f1
def signalScan():
    scan = "{0}".format(branchNames[0])
    for i in branchNames[1:-1]:
        scan += ":{0}".format(i)
    scan += ":{0}".format(branchNames[12])
    

In [7]:
def radiusCheck(jdata, mc_data, mc_data_hig):
    count = 0
    l = np.amax(jdata[:,0])
    l = l.astype(int)
    for i in range(l):
        eventID = np.where(jdata[:, 0] == i)
        jeta = jdata[eventID, 4][0]
        jphi = jdata[eventID, 5][0]
        for jEta, jPhi in zip(jeta, jphi):
            if jEta != -500 and jPhi != -500:
                #print jEta, jPhi
                deltaEta = mc_data_hig[eventID, 4] - jEta 
                deltaPhi = mc_data_hig[eventID, 5] - jPhi
                deltaR   = np.square(deltaEta) + np.square(deltaPhi)
                #print deltaR
                check = np.where(deltaR[0] <= .2)
                count += 1
                if count % 500 == 0:
                    print check[0]
                #print count, deltaR[check]

In [18]:
radiusCheck(jetdata,mcdata, higgsP)

NameError: name 'jetdata' is not defined

In [ ]:
def calRatio():

In [ ]:
def countTrack():

In [ ]:
def higgsChildren(higgsA, jetA, mcdataA):
    higgsChildRows = np.hstack( higgsA[:,0] )
    childP_j = jetA[ higgsChildRows ]
    childP_mc = mcdataA[ higgsChildRows ]
    higChild = np.column_stack([ childP_j, childP_mc ])
    return higChild

In [22]:
def main():
    sigArray = hv.loader("signal")
    jetdata  = vert_jetTrk(sigArray)
    mcdata   = vert_mcdata(sigArray)
    mcdata_hig = locateHiggsEvents(mcdata)
    print mcdata_hig
    #higChild = higgsChildren(higgs_mcdata, jetdata, mcdata)

In [ ]:
p

In [23]:
main()

float64
<type 'numpy.ndarray'>
shape: (203640, 9)
entries: 1832760

object
<type 'numpy.ndarray'>
shape: (6328372, 6)
entries: 37970232

[[array([118, 119], dtype=int32) 36 0.08018578588962555 1.1115773916244507 -2.502485752105713
  2.314896821975708]
 [array([120, 121], dtype=int32) 36 0.08018578588962555 1.1115773916244507 -2.822516918182373
  -1.628440260887146]
 [array([697, 698], dtype=int32) 36 0.07148713618516922 1.113144040107727 -0.0334453359246254
  -1.5927108526229858]
 ..., 
 [array([171, 172], dtype=int32) 36 0.03184041753411293 1.1201077699661255 -0.8341189622879028
  1.7432770729064941]
 [array([676, 677], dtype=int32) 36 0.13462238013744354 1.0535780191421509 -2.750379800796509
  0.5363646149635315]
 [array([678, 679], dtype=int32) 36 0.13462238013744354 1.0535780191421509 -1.8100703954696655
  -2.087846517562866]]


In [8]:
sigArray = hv.loader("signal")

In [14]:
%%timeit
jetdata  = vert_jetTrk(sigArray)
mcdata   = vert_mcdata(sigArray)
higgsP   = locateHiggsEvents(mcdata)

float64
<type 'numpy.ndarray'>
shape: (203640, 9)
entries: 1832760

object
<type 'numpy.ndarray'>
shape: (6328372, 6)
entries: 37970232

float64
<type 'numpy.ndarray'>
shape: (203640, 9)
entries: 1832760

object
<type 'numpy.ndarray'>
shape: (6328372, 6)
entries: 37970232

float64
<type 'numpy.ndarray'>
shape: (203640, 9)
entries: 1832760

object
<type 'numpy.ndarray'>
shape: (6328372, 6)
entries: 37970232

float64
<type 'numpy.ndarray'>
shape: (203640, 9)
entries: 1832760

object
<type 'numpy.ndarray'>
shape: (6328372, 6)
entries: 37970232

1 loop, best of 3: 6.17 s per loop


In [15]:
childP = np.hstack( locateHiggsEvents(mcdata)[:, 0] )
print childP

NameError: name 'mcdata' is not defined

In [ ]:
print mcdata[:, 0]
print 
print "---------" 
print
print locateHiggsEvents(mcdata).shape


In [ ]:
orignallen = len(jetdata)
higgsCut = len()
cutFlow = { 'Start' : orignallen,  }
print cutFlow['Start']

In [ ]:
print higgsP

In [ ]:
print jetdata[50:80, 4]

In [19]:
a = np.where(jetdata[:, 0] == 2)
jetdata[a, 5]

NameError: name 'jetdata' is not defined